In [83]:
%%file formulation.lp

#const intlen = 32.

register(eax;ebx;ecx;edx;esi;edi;edp;esp;eip;imm;cmp).
flag(n;z;p).
apitype_value(get_something).
apitype_alloc(alloc_something).
pos(0..intlen-1).
time(0..maxtime).
pc(0..maxpc).
flg(0, z).
    
:- flg(T, F1), flg(T, F2), F1 != F2, flag(F1), flag(F2), time(T).

%% Generalizations
cmp(T) :- time(T), pcat(T, PC), cmp_imm(PC, _, _).
imm(T) :- time(T), pcat(T, PC), cmp_imm(PC, _, _).
    
%% Registers maintain value unless acted upon
regbit(T, Reg, Pos) :-
    time(T),
    register(Reg),
    pos(Pos),
    regbit(T-1, Reg, Pos),
    pcat(T, PC),
    not end(PC),
    not sub(PC, _, _, Reg),
    Reg != eip,
    Reg != eax,
    Reg != imm.
regbit(T, imm, Pos) :-
    time(T),
    pos(Pos),
    regbit(T-1, imm, Pos),
    pcat(T, PC),
    not end(PC),
    not imm(T).
regbit(T, eax, Pos) :-
    time(T),
    pos(Pos),
    regbit(T-1, eax, Pos),
    pcat(T, PC),
    not end(PC),
    not apicall(PC, _).
    
%% PC advances each time step
regbit(T+1, eip, 0) :-
    time(T),
    not regbit(T, eip, 0).
regbit(T+1, eip, Pos) :-
    time(T),
    pos(Pos),
    Pos > 0,
    regbit(T, eip, Pos),
    regbit(T+1, eip, Pos-1).
regbit(T+1, eip, Pos) :-
    time(T),
    pos(Pos),
    Pos > 0,
    regbit(T, eip, Pos),
    not regbit(T, eip, Pos-1).
regbit(T+1, eip, Pos) :-
    time(T),
    pos(Pos),
    Pos > 0,
    not regbit(T, eip, Pos),
    not regbit(T+1, eip, Pos-1),
    regbit(T, eip, Pos-1).
    

% Comparisons    

%% Store value from cmp_imm in imm
regbit(T, imm, Pos) :-
    time(T),
    pos(Pos),
    pcat(T, PC),
    cmp_imm(PC, _, Pos).

%% Reg - imm -> cmp
sub(PC, Reg, imm, cmp) :- cmp_imm(PC, Reg, _).

%% Subtraction
regbit(T, Res, Pos) :-
    time(T),
    register(Res),
    pos(Pos),
    pcat(T, PC),
    sub(PC, Reg1, Reg2, Res),
    register(Reg1),
    register(Reg2),
    regbit(T, Reg1, Pos),
    not regbit(T, Reg2, Pos),
    not carry(T, Pos).
regbit(T, Res, Pos) :-
    time(T),
    register(Res),
    pos(Pos),
    pcat(T, PC),
    sub(PC, Reg1, Reg2, Res),
    register(Reg1),
    register(Reg2),
    not regbit(T, Reg1, Pos),
    regbit(T, Reg2, Pos),
    not carry(T, Pos).
regbit(T, Res, Pos) :-
    time(T),
    register(Res),
    pos(Pos),
    pcat(T, PC),
    sub(PC, Reg1, Reg2, Res),
    register(Reg1),
    register(Reg2),
    regbit(T, Reg1, Pos),
    regbit(T, Reg2, Pos),
    carry(T, Pos).
regbit(T, Res, Pos) :-
    time(T),
    register(Res),
    pos(Pos),
    pcat(T, PC),
    sub(PC, Reg1, Reg2, Res),
    register(Reg1),
    register(Reg2),
    not regbit(T, Reg1, Pos),
    not regbit(T, Reg2, Pos),
    carry(T, Pos).
carry(T, Pos+1) :-
    time(T),
    pos(Pos),
    pcat(T, PC),
    sub(PC, Reg1, Reg2, Res),
    register(Reg1),
    register(Reg2),
    not regbit(T, Reg1, Pos),
    regbit(T, Reg2, Pos).
carry(T, Pos+1) :-
    time(T),
    pos(Pos),
    pcat(T, PC),
    sub(PC, Reg1, Reg2, Res),
    register(Reg1),
    register(Reg2),
    regbit(T, Reg1, Pos),
    regbit(T, Reg2, Pos),
    carry(T, Pos).
carry(T, Pos+1) :-
    time(T),
    pos(Pos),
    pcat(T, PC),
    sub(PC, Reg1, Reg2, Res),
    register(Reg1),
    register(Reg2),
    not regbit(T, Reg1, Pos),
    not regbit(T, Reg2, Pos),
    carry(T, Pos).
    
%% API Calls: single value
0 { regbit(T, eax, Pos) } 1 :-
    time(T),
    pos(Pos),
    pcat(T, PC),
    apicall(PC, Call),
    apitype_value(Call).
    
%% Flags
flg(T, n) :- 
    cmp(T),
    carry(T, intlen).
flg(T, z) :-
    cmp(T),
    not regbit(T, cmp, _),
    not carry(T, intlen).
flg(T, p) :-
    cmp(T),
    regbit(T, cmp, _),
    not carry(T, intlen).

Overwriting formulation.lp


In [84]:
%%file instance-6.lp

#const maxtime = 6.
#const maxpc = 5.

apicall(1, get_something).
cmp_imm(2, eax, 0).
jge(3, 5).
eicar(4).
end(maxpc).

Overwriting instance-6.lp


In [88]:
%%file suppress-warnings.lp

apicall(0, get_something) :- a=b.
cmp_imm(0, eax, zero) :- a=b.
jlt(0, 0) :- a=b.
jle(0, 0) :- a=b.
jeq(0, 0) :- a=b.
jne(0, 0) :- a=b.
jge(0, 0) :- a=b.
jgt(0, 0) :- a=b.
eicar(0) :- a=b.

Writing suppress-warnings.lp


In [85]:
%%file gen-pcat.py

import sys

bits = int(sys.argv[1])
maxpc = int(sys.argv[2])

with open('pcat' + str(bits) + '.lp', 'w') as f:
    f.write('toolong(T) :-\n')
    f.write('    time(T),\n')
    f.write('    regbit(T, eip, Pos),\n')
    f.write('    pos(Pos),\n')
    f.write('    Pos >= ' + str(maxpc.bit_length()) + '.\n\n')
    for i in range(0, maxpc + 1):
        f.write('pcat(T, ' + str(i) + ') :-\n')
        f.write('    time(T),\n')
        f.write('    pc(PC),\n')
        for j in range(0, maxpc.bit_length()):
            f.write('    ' + ('' if i&(1<<j) else 'not ') + 'regbit(T, eip, ' + str(j) + '),\n')
        f.write('    not toolong(T).\n')

Overwriting gen-pcat.py


In [86]:
%run gen-pcat 32 5

In [90]:
!clingo formulation.lp instance-6.lp pcat32.lp suppress-warnings.lp -c intlen=32 --time-limit=60

clingo version 5.1.0
Reading from formulation.lp ...
Solving...
Answer: 1
apicall(1,get_something) cmp_imm(2,eax,0) pos(0) pos(1) pos(2) pos(3) pos(4) pos(5) pos(6) pos(7) pos(8) pos(9) pos(10) pos(11) pos(12) pos(13) pos(14) pos(15) pos(16) pos(17) pos(18) pos(19) pos(20) pos(21) pos(22) pos(23) pos(24) pos(25) pos(26) pos(27) pos(28) pos(29) pos(30) pos(31) time(0) time(1) time(2) time(3) time(4) time(5) time(6) pc(0) pc(1) pc(2) pc(3) pc(4) pc(5) flag(n) flag(z) flag(p) flg(0,z) register(eax) register(ebx) register(ecx) register(edx) register(esi) register(edi) register(edp) register(esp) register(eip) register(imm) register(cmp) end(5) sub(2,eax,imm,cmp) apitype_value(get_something) apitype_alloc(alloc_something) pcat(0,0) regbit(1,eip,0) regbit(2,eip,1) regbit(3,eip,0) regbit(3,eip,1) regbit(4,eip,2) regbit(5,eip,0) regbit(5,eip,2) regbit(6,eip,1) regbit(6,eip,2) regbit(7,eip,0) regbit(7,eip,1) pcat(1,1) pcat(3,3) pcat(2,2) regbit(7,eip,2) regbit(3,imm,0) regbit(2,imm,0) imm(2) cm